### Description
Web Scraping tool made using BeautifulSoup. It prompts the user for a video game title and the number of pages it should scrape. Once done it will prompt for another title or the user can also type quit to stop it. The collected data is stored in a csv file named output in the same folder as the code. The data stored in each row is the name of the vidoe game, username of the reviewer, the date it was reviewed, the score given, and the written review. 

Limitation / Issues of the code
-  it only looks for PC game reviews. It should be easy to tweak the URL variable for it to look for video games on different platforms.
- naming the video game has to be specific; 'diablo iv' works but 'diablo 4' does not. This issue will not throw an error the prompt will just appear again without anything being collected. I recommend visiting metacritic.com to see how it was written there.
- Due to labeling in the html, I wasn't able to find away to scrape the reviews that appear first and last in the web pages.
- Foreign languages will be scrapped too. I'm looking into how to only collect english reviews.

In [19]:
# Code adapted from https://towardsdatascience.com/web-scraping-metacritic-reviews-using-beautifulsoup-63801bbe200e
from bs4 import BeautifulSoup
import requests
import pandas as pd

review_dict = {'video game':[],'username':[], 'date':[],'score':[],'review':[]}
while True: # While there are still reviews to collect
    video_game = input().lower() # Type in the name of a video game
    if (video_game == 'quit'): break # Stops collecting reviews if prompted to quit
    video_game = video_game.replace(' ', '-').replace(':','').lower() 
    
    print("What are the number of pages?")
     # Might not be relevant as giving number of pages exceeding one on website doesn't throw errors
    num_pages = int(input())
    
    for page in range(0,num_pages):
        URL = "https://www.metacritic.com/game/pc/"+ video_game + "/user-reviews?sort-by=date&num_items=100&page="+str(page)
        user_agent = {'User-agent': 'Mozilla/5.0'}
        response = requests.get(URL, headers = user_agent)
        soup = BeautifulSoup(response.text, 'html.parser')

        for review in soup.find_all('li', class_='review user_review'):
            review_dict['video game'].append(video_game)
            if review.find('span', class_='blurb blurb_expanded'):
                review_dict['review'].append( # Review of game
                    review.find('span', class_='blurb blurb_expanded').text)
                review_dict['username'].append( # Username of reviewer
                    review.find('div', class_='review_critic').find('a').text)
                review_dict['date'].append( # Date of the review
                    review.find('div', class_='date').text)
                review_dict['score'].append( # Score given
                    review.find('div', class_='review_grade').find('div').text)
            else:
                review_dict['review'].append(
                    review.find('div',class_='review_body').find('span').text) 
                review_dict['username'].append(
                    review.find('div', class_='review_critic').find('a').text)
                review_dict['date'].append(
                    review.find('div', class_='date').text)
                review_dict['score'].append(
                    review.find('div', class_='review_grade').find('div').text)
reviews = pd.DataFrame(review_dict)
reviews['score'] = [int(x) for x in reviews['score']]
reviews['rating'] = ['Negative' if x < 5 else 'Positive' if x > 7
                     else 'Average' for x in reviews['score']]
reviews.to_csv('output.csv')

Resident Evil 4
What are the number of pages?
10
Resident Evil 2
What are the number of pages?
10
Resident Evil 3
What are the number of pages?
10
Resident Evil Village
What are the number of pages?
10
Resident Evil 7 Biohazard
What are the number of pages?
10
quit
